In [1]:
import jellyfish
import pandas as pd
from pytrends.request import TrendReq

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

In [3]:
pytrends.trending_searches()

,0
0,CNN live
1,Pi Day
2,Trish Regan
3,Conor McGregor
4,Ronaldinho
5,Frozen 2 Disney
6,Westworld
7,Spain
8,Xavier Rhodes
9,Jack Ma


In [4]:
pytrends.trending_searches()[0][0]

'CNN live'

In [5]:
first_trend = pytrends.trending_searches()[0][0]
second_trend = pytrends.trending_searches()[0][1]
third_trend = pytrends.trending_searches()[0][2]
fourth_trend = pytrends.trending_searches()[0][3]
fifth_trend = pytrends.trending_searches()[0][4]

In [6]:
second_trend

'Pi Day'

In [7]:
fifth_trend

'Ronaldinho'

realizing that sometimes two trending searches could be highly related to one another. case in point when run on 03/14/2020, the second trend was `Pi Day` and the fifth trend was `Pi Day 2020`

In [8]:
for i in range(len(pytrends.trending_searches()))[:5]:
    print(pytrends.trending_searches()[0][i])

CNN live
Pi Day
Trish Regan
Conor McGregor
Ronaldinho


Using jellyfish we can calculate string similarity using the metric : $argmin$

In [10]:
jellyfish.levenshtein_distance(second_trend, fifth_trend)

5

In [11]:
jellyfish.levenshtein_distance(first_trend, second_trend)

7

In [12]:
jellyfish.damerau_levenshtein_distance(second_trend, fifth_trend)

5

In [13]:
jellyfish.jaro_distance(second_trend, fifth_trend)

0.8484848484848485

In [14]:
jellyfish.jaro_distance(first_trend, second_trend)

0.4305555555555555

In [15]:
jellyfish.jaro_distance('ip day', 'pi day')

0.9444444444444445

In [16]:
jellyfish.jaro_distance('2020 pi day', 'pi day')

0.41919191919191917

In [17]:
jellyfish.damerau_levenshtein_distance('pi day', 'pi day')

0

In [18]:
jellyfish.match_rating_comparison(second_trend, 'pye day') == True

True

In [19]:
trends = pytrends.trending_searches()
trends.columns = ['trend']

In [21]:
dld = lambda x, y: jellyfish.damerau_levenshtein_distance(x, y)

In [23]:
distance_list = [i for i in trends['trend'].values]

In [24]:
dp = distance_list[0]

In [25]:
distance_lists = []
for i in range(19):
    dp = distance_list[i]
    dps_similarity_index = list(enumerate([jellyfish.damerau_levenshtein_distance(dp, j) for j in distance_list]))
    dps_similarity_index.sort(key=lambda x:x[1])
    distance_lists.append(dps_similarity_index)

In [26]:
for i in distance_lists:
    print( i[0][0])
    print("also,", i[1][1])
   #if i[0][0] == i[0][0]:
   #    print(i)

0
also, 7
1
also, 5
2
also, 8
3
also, 8
4
also, 5
5
also, 11
6
also, 9
7
also, 10
8
also, 6
9
also, 11
10
also, 7
11
also, 11
12
also, 11
13
also, 15
14
also, 27
15
also, 9
16
also, 16
17
also, 14
18
also, 15


In [37]:
trends_trans = pd.DataFrame(trends.T).reset_index(drop=True)

In [38]:
trends_trans

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,CNN live,Pi Day,Trish Regan,Ronaldinho,Pi Day 2020,Conor McGregor,Xavier Rhodes,Frozen2 Disney,Jack Ma,Andrew Gillum,Bill Gates,Friday the 13th,Jay Electronica,Disney World closures,Coronavirus disease 2019 Prevention,Ivanka Trump,Animal Crossing Switch,Lost Girls Netflix,Illinois school closings,Grace VanderWaal


In [76]:
dfs = []
for i in range(len(trends)): 
    for j in trends_trans:
        counter = j + 1
        # calculate damerau levenshtein distance between two strings
        distance = jellyfish.damerau_levenshtein_distance(trends['trend'][i], trends_trans[j][0])
        jaro_distance = jellyfish.jaro_distance(trends['trend'][i], trends_trans[j][0])
        # create a temp dataframe then append to a list of all dataframes
        temp_df = pd.DataFrame({'firststring':trends['trend'][i], 
                                'secondstring':trends_trans[j][0], 
                                'thedistance':distance,
                               'jaro_distance':jaro_distance}, index=[counter])
        dfs.append(temp_df)

In [77]:
ldistance_df = pd.concat(dfs)

In [81]:
ldistance_df[ldistance_df['jaro_distance'] > .75]

,firststring,secondstring,thedistance,jaro_distance
1,CNN live,CNN live,0,1.000000
2,Pi Day,Pi Day,0,1.000000
5,Pi Day,Pi Day 2020,5,0.848485
3,Trish Regan,Trish Regan,0,1.000000
4,Ronaldinho,Ronaldinho,0,1.000000
2,Pi Day 2020,Pi Day,5,0.848485
5,Pi Day 2020,Pi Day 2020,0,1.000000
6,Conor McGregor,Conor McGregor,0,1.000000
7,Xavier Rhodes,Xavier Rhodes,0,1.000000
8,Frozen2 Disney,Frozen2 Disney,0,1.000000


In [83]:
ldistance_df[(ldistance_df['thedistance'] <6) & (ldistance_df['firststring'] != ldistance_df['secondstring'])]

,firststring,secondstring,thedistance,jaro_distance
5,Pi Day,Pi Day 2020,5,0.848485
2,Pi Day 2020,Pi Day,5,0.848485


## PyTrends

In [10]:
trends = pytrends.trending_searches()
trends.columns = ['trend']

In [11]:
trends_trans = pd.DataFrame(trends.T).reset_index(drop=True)